In [5]:
pip install pycairo

  Using cached pycairo-1.19.1.tar.gz (205 kB)
  Created wheel for pycairo: filename=pycairo-1.19.1-cp37-cp37m-macosx_10_7_x86_64.whl size=93498 sha256=9b4ca44d551df9018e79ffc357f3a4727e7f447bb3ab11119655ba7e83a524c2
  Stored in directory: /Users/leondowner/Library/Caches/pip/wheels/82/aa/e4/b6c8ba84a111a5485e7a426b030f03b77765cdc2567927ea96
Successfully built pycairo
You should consider upgrading via the '//anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd 
import string
import numpy as np

import cairo

    
import sys
import argparse
import copy
import math
import random
from textblob  import  TextBlob


In [103]:
data = pd.read_csv('watchtower_lyrics.csv')

In [104]:
data.head()

,lyric
0,There must be some kind of way outta here
1,Said the joker to the thief
2,There's too much confusion
3,I can't get no relief
4,"Business men, they drink my wine"


In [105]:
pol = lambda x: TextBlob(x).sentiment.polarity 
sub = lambda x: TextBlob(x).sentiment.subjectivity  

In [121]:
data['polarity'] = data['lyric'].apply(pol)
data['subjectivity'] = data['lyric'].apply(sub)
data['totalwords'] = data['lyric'].str.split().str.len()
data['pol_score'] = data['polarity'] * data['totalwords']


data['polarity'] = data['polarity'].astype(float)

def process(row):
            polarity = row['polarity']
            if polarity >0: return "0, 153, 0" #green
            if polarity ==0: return "0, 102, 255" #blue
            if polarity  <0: return "255, 0, 0" #red

data['colour'] = data.apply(process,axis=1)

#data['colour'] = pd.to_numeric(data['colour'])


In [122]:
data.head(30)

,lyric,polarity,subjectivity,totalwords,pol_score,colour
0,There must be some kind of way outta here,0.600,0.900,9,5.400,"0, 153, 0"
1,Said the joker to the thief,0.000,0.000,6,0.000,"0, 102, 255"
2,There's too much confusion,0.200,0.200,4,0.800,"0, 153, 0"
3,I can't get no relief,0.000,0.000,5,0.000,"0, 102, 255"
4,"Business men, they drink my wine",0.000,0.000,6,0.000,"0, 102, 255"
5,Plowmen dig my earth,0.000,0.000,4,0.000,"0, 102, 255"
6,None will level on the line,0.000,0.000,6,0.000,"0, 102, 255"
7,Nobody offered his word,0.000,0.000,4,0.000,"0, 102, 255"
8,"Hey, hey",0.000,0.000,2,0.000,"0, 102, 255"
9,No reason to get excited,0.375,0.750,5,1.875,"0, 153, 0"


In [126]:


#float_gen = lambda a, b: random.uniform(a, b)

#colors = []
#for i in range(15):
#    colors.append((float_gen(.4, .75), float_gen(.4, .75), float_gen(.4, .75)))


colors = []

colors.append(data.colour.str.split(', ').map(float).unique())



#colors.append(float(data.colour.unique()))

              #.strip('[]'))
#colors.append(np.vectorize(data.colour.unique()))

def octagon(x_orig, y_orig, side):
    x = x_orig
    y = y_orig
    d = side / math.sqrt(2)

    oct = []

    oct.append((x, y))

    x += side
    oct.append((x, y))

    x += d
    y += d
    oct.append((x, y))

    y += side
    oct.append((x, y))

    x -= d
    y += d
    oct.append((x, y))

    x -= side
    oct.append((x, y))

    x -= d
    y -= d
    oct.append((x, y))

    y -= side
    oct.append((x, y))

    x += d
    y -= d
    oct.append((x, y))

    return oct

def deform(shape, iterations, variance):
    for i in range(iterations):
        for j in range(len(shape)-1, 0, -1):
            midpoint = ((shape[j-1][0] + shape[j][0])/2 + float_gen(-variance, variance), (shape[j-1][1] + shape[j][1])/2 + float_gen(-variance, variance))
            shape.insert(j, midpoint)
    return shape

   
def main(**kwargs):
    """parser = argparse.ArgumentParser()
    parser.add_argument("--width", default=1000, type=int)
    parser.add_argument("--height", default=1500, type=int)
    parser.add_argument("-i", "--initial", default=120, type=int)
    parser.add_argument("-d", "--deviation", default=50, type=int)
    parser.add_argument("-bd", "--basedeforms", default=1, type=int)
    parser.add_argument("-fd", "--finaldeforms", default=3, type=int)
    parser.add_argument("-mins", "--minshapes", default=20, type=int)
    parser.add_argument("-maxs", "--maxshapes", default=25, type=int)
    parser.add_argument("-sa", "--shapealpha", default=.02, type=float)
    args = parser.parse_args()
    """
    kwargs = {"width": 1000,
            "height": 1000,
            "initial": 120,
            "deviation": 50,
            "basedeforms": 1,
            "finaldeforms": 3,
            "minshapes": 20,
            "maxshapes": 25,
            "shapealpha":.02}


    
    #main(**kwargs)
    
    width = kwargs['width']
    height = kwargs['height']
    initial = kwargs['initial']
    deviation = kwargs['deviation']

    basedeforms = kwargs['basedeforms']
    finaldeforms = kwargs['finaldeforms']

    minshapes = kwargs['minshapes']
    maxshapes = kwargs['maxshapes']

    shapealpha = kwargs['shapealpha']

    ims = cairo.ImageSurface(cairo.FORMAT_ARGB32, width, height)
    cr = cairo.Context(ims)

    cr.set_source_rgb(.9, .9, .9)
    cr.rectangle(0, 0, width, height)
    cr.fill()

    cr.set_line_width(1)

    for p in range(-int(height*.2), int(height*1.2), 60):
        cr.set_source_rgba(random.choice(colors)[0], random.choice(colors)[1], random.choice(colors)[2], shapealpha)

        shape = octagon(random.randint(-100, width+100), p, random.randint(100, 300))
        baseshape = deform(shape, basedeforms, initial)

        for j in range(random.randint(minshapes, maxshapes)):
            tempshape = copy.deepcopy(baseshape)
            layer = deform(tempshape, finaldeforms, deviation)

            for i in range(len(layer)):
                cr.line_to(layer[i][0], layer[i][1])
            cr.fill()

    ims.write_to_png('' + str(int(random.randint(0, 500))) + '.png')

if __name__ == "__main__":
    main()

TypeError: float() argument must be a string or a number, not 'list'

In [52]:
kwargs = {"width": 1000,
            "height": 1000,
            "initial": 120,
            "deviation": 50,
            "basedeforms": 1,
            "finaldeforms": 3,
            "minshapes": 20,
            "maxshapes": 25,
            "shapealpha":.02}

In [53]:
print(kwargs)

{'width': 100, 'height': 1000, 'initial': 120, 'deviation': 50, 'basedeforms': 1, 'finaldeforms': 3, 'minshapes': 20, 'maxshapes': 25, 'shapealpha': 0.02}


In [54]:
kwargs['width']

100

In [55]:
width = kwargs['width']
print(width)

100


In [63]:
print(colors)

[(0.41592624685088336, 0.576017069427655, 0.6321665299325263), (0.6029413282453896, 0.603685315840183, 0.5277602191457426), (0.590950584903762, 0.4813859981863262, 0.5208824725321652), (0.4345009093811144, 0.6673092010403858, 0.6102383187651381), (0.6912299033190688, 0.4513711366642734, 0.5253521445437327), (0.500288106302018, 0.6020625647779603, 0.49842993863907514), (0.6398968287460131, 0.6922946104556948, 0.5945728865807012), (0.5800527019321726, 0.5186771228434401, 0.6721934408906454), (0.6629420847848423, 0.6877077422033047, 0.658408277680314), (0.6403816057199522, 0.6458558816578657, 0.679249144127651), (0.6981418609087614, 0.6848881255788364, 0.6199405444851657), (0.7267624367353926, 0.5163355542638541, 0.4256372886732189), (0.5048992950042869, 0.6953911855355166, 0.7213019390258588), (0.6647786346451967, 0.5639609167789051, 0.5900443106628914), (0.48681259230152163, 0.47032485211732444, 0.48675084648791955)]


In [88]:
print(colors)

[]


In [124]:
print((data.colour.unique()))

['0, 153, 0' '0, 102, 255' '255, 0, 0']


In [90]:
print((data.colour.unique()))

['0, 153, 0' '0, 102, 255' '255, 0, 0']


In [74]:
print (str(data.colour.unique()).strip('[]'))

'0, 153, 0' '0, 102, 255' '255, 0, 0'


In [91]:
data = {'col1':  ['abc', 'def','ghi','jkl'],
        'col2': ['0, 153, 0', '0, 153, 0', '0, 102, 255','255, 0, 0']
        }

df = pd.DataFrame (data, columns = ['col1','col2'])

In [92]:
l = df.col2.str.split(', ').map(tuple).unique()

In [93]:
print(l)

[('0', '153', '0') ('0', '102', '255') ('255', '0', '0')]


In [116]:
l2 = l.str.split(', ')

AttributeError: 'numpy.ndarray' object has no attribute 'str'